# Validation of Pyxel's CCD effects for MARVEL

The following values are from the STA-1600 CCD model.  

- Area: $10.3 \times 10.3$k 9-µm pixels ($95\times95$ mm image area) 

$\textbf{Two read modes exist:}$

Both modes uses a output amplifier sensitivity of 7 $\mu$V/e:

Fast readout (50 kHz):                                                       
- Readout noise: 2 e-                                                                            
- Gain: 65.8 $\mu$V/DN $\rightarrow$ 15198 DN/V $\rightarrow$ 9.4 e/DN                                                                        
                                                                             
Slow readout (1 MHz):                                                                            
- Readout noise: 5 e-                                                                             
- Gain: 21.9 $\mu$V/DN $\rightarrow$ 45662 DN/V $\rightarrow$ 3.0 e/DN

In [13]:
import os
import yaml
import shutil
import pyxel
import pyechelle
import numpy as np
from astropy.io import fits 

# Random number generator and fixed seed to reproduce results
rng = np.random.default_rng(12345)

# Print version
print(f"Pyxel     v. {pyxel.__version__}")
print(f"PyEchelle v. {pyechelle.__version__}")

Pyxel     v. 1.7
PyEchelle v. 0.3.2


## Configure Pyxel for MARVELsim

We will in the following only use the exposure mode of Pyxel and load in PyEchelle images with pixel units in electron flux.

**Note on exposure time and time scale**

Setting a longer exposure time should always result in a larger count. We can imagine the image we provide as flux. By default it will be interpreted as flux per 1 second (the time scale). If we overwrite the time scale in the model to 10 s, it will be the flux per 10 s, so the count will decrease if we leave the exposure time the same. This way you can load an image with a known exposure time for example and have a correct flux.

### Output path

Unfortunately Pyxel do not have any clever way to secure that the output folder is set through the attributes. This means we need to open the YAML file ourself and correct the filepath to where we want our simulations to be stored. Notice that the following is already secured while running MARVELsim while parsing `-o </path/to/outdir>`. The following is a validation of the code:

In [14]:
# File paths to input and output YAML file
ifile = os.getcwd() + "/../inputfiles/inputfile_marvel.yaml"
ofile = os.getcwd() + "/output/inputfile_marvel.yaml" 

# First copy YAML to avoid overwriting the original file
shutil.copy2(ifile, ofile)

# Load the data within the YAML file
stream = open(ofile, 'r')
data   = yaml.full_load(stream)

# Alter the output file location
data['exposure']['outputs']['output_folder'] = "output"

# Overwrite YAML file
with open(ofile, 'w') as yaml_file:
    yaml_file.write(yaml.dump(data, default_flow_style=False))

### Initialise Pyxel

In [15]:
# Load the MARVEL input YAML file
config = pyxel.load(ifile)

# Setup configurations
exposure = config.exposure
detector = config.ccd_detector
pipeline = config.pipeline

### CCD parameters

We here set all the parameters needed for the `detector object` reflecting the MARVEL CCD properties. Notice this is only for illustration on how to do this since this parameter space is already configured in the input YAML file.

In [16]:
# Quantum efficientcy
detector.characteristics.quantum_efficiency = 0.9

# Amplifier readout sensitivity [V/e-]
detector.characteristics.charge_to_volt_conversion = 7.0e-6

# Gain of output amplifier [V/V] -> User defined!
detector.characteristics.pre_amplification = 12

# Bit resolution [bits]
detector.characteristics.adc_bit_resolution = 16

# Output DC Level [V]
detector.characteristics.adc_voltage_range = [0., 16]

# Full-well capacity [e-]
detector.characteristics.full_well_capacity = 800000

In [17]:
# Load a zeros array
pipeline.charge_generation.load_charge.arguments.filename = 'flat_pyechelle_10s.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 5
exposure.readout.times = [5]

# Configure dark rate
pipeline.charge_generation.simple_dark_current.arguments.dark_rate = 0.01

# Disable CosmiX
pipeline.photon_generation.cosmix.enabled = True

# Load the model of incident proton hits
pipeline.photon_generation.cosmix.arguments.spectrum_file  = os.getcwd() + '/../inputfiles/proton_L2_solarMax_11mm_Shielding.txt

/tmp/ipykernel_24391/816139352.py:10: DeprecationWarning: Group 'photon_generation' is deprecated and will be removed in version 2.0. Use group 'photon_collection' instead.
  pipeline.photon_generation.cosmix.enabled = True
/tmp/ipykernel_24391/816139352.py:13: DeprecationWarning: Group 'photon_generation' is deprecated and will be removed in version 2.0. Use group 'photon_collection' instead.
  pipeline.photon_generation.cosmix.arguments.spectrum_file  = os.getcwd() + '/../inputfiles/proton_L2_solarMax_11mm_Shielding.txt'


In [19]:
# Run simulations and plot
results = pyxel.exposure_mode(exposure, detector, pipeline)
results

Cosmix:   0%|          | 0/5 [00:00<?, ? particle/s]

<xarray.Dataset>
Dimensions:       (readout_time: 1, y: 10560, x: 10560)
Coordinates:
  * readout_time  (readout_time) int64 5
  * y             (y) int64 0 1 2 3 4 5 ... 10554 10555 10556 10557 10558 10559
  * x             (x) int64 0 1 2 3 4 5 ... 10554 10555 10556 10557 10558 10559
Data variables:
    image         (readout_time, y, x) uint32 3426 3424 3426 ... 3426 3427 3427
    signal        (readout_time, y, x) float64 0.8364 0.8359 ... 0.8366 0.8365
    pixel         (readout_time, y, x) float64 9.4e+03 9.395e+03 ... 9.401e+03
Attributes:
    pyxel version:  1.7
    running mode:   Exposure

In [ ]:
# Displaying the detector with the code below crashes Jupyter because the image area is so huge!
# pyxel.display_detector(detector)

# Instead we 

### Bias image

Currently (Marts 2022) Pyxel do not have an option to automatically determine the bias level given the CCD parameters stated within the class `ccd_detector`. For now we use PyEchelle's CCD flags called `--bias <VALUE>` to include the bias level and additionally `--read_noise <VALUE>` to add a random noise realization of the readout.     

If we assume that the bias level to be 1000 ADU, this transform fast-readout bias level of 9400 e- and a slow-readout bias level of 3000 e-. The RMS readout noise is:

- Fast readout RMS noise: 2.5 e- (nominal) and 4.0 e- (maximum)  
- Slow readout RMS noise: 5.0 e- (nominal) and 7.0 e- (maximum)  

Thus the bias can be constructed normal distributed image with a mean of the bias level and a standard deviation comparable to the RMS noise level. The RMS and standard deviation are identical when we talk about the residual noise since the mean has been subtracted. 

In [ ]:
# Generate bias image
bias = rng.normal(3000, scale=5.0, size=(500,500)).astype(int)  

# Save above bias images
hdul = fits.HDUList([fits.PrimaryHDU(bias)])
hdul.writeto('bias_slow.fits', overwrite=True)

bias

In [ ]:
# Load a bias image
pipeline.charge_generation.load_charge.arguments.filename = 'bias_slow.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 1
exposure.readout.times = [1]

# Disable cosmix to
pipeline.photon_generation.cosmix.enabled = False

# Run simulations and plot (select Array = "image")
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

In [ ]:
bias = rng.normal(9400, scale=2.5, size=(500,500)).astype(int)  

# Save above bias images
hdul = fits.HDUList([fits.PrimaryHDU(bias)])
hdul.writeto('bias_fast.fits', overwrite=True)

bias

In [ ]:
# Load a bias image
pipeline.charge_generation.load_charge.arguments.filename = 'bias_fast.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 1
exposure.readout.times = [1]

# Disable cosmix to
pipeline.photon_generation.cosmix.enabled = False

# Run simulations and plot (select Array = "image")
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

### Dark current image

Dark Signal output signal is caused by thermally generated electrons. Dark signal is a linear function of integration time and an exponential function of chip temperature. We here assume a constant CCD chip temperature of 200 K (specified in the input YAML file). For the given CCD the dark current is:

- Nom. : 3.0 e/pix/hour @ -100C
- Max. : 5.0 e/pix/hour @ -100C



In [ ]:
# Create a 500 x 500 pixel array of zeros
zero = np.zeros((500,500))

# Save above bias image
hdul = fits.HDUList([fits.PrimaryHDU(zero)])
hdul.writeto('zeros.fits', overwrite=True)

zero

In [ ]:
# Load a zeros array
pipeline.charge_generation.load_charge.arguments.filename = 'zeros.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 900
exposure.readout.times = [900]

# Configure dark rate
pipeline.charge_generation.simple_dark_current.arguments.dark_rate = 0.01

# Exclude cosmic rays to visualize dark current
pipeline.photon_generation.cosmix.enabled = False

# Run simulations and plot
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

### Flat-field spectrum

In [ ]:
# Load a zeros array
pipeline.charge_generation.load_charge.arguments.filename = 'flat_pyechelle_5s.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 5
exposure.readout.times = [5]

# Configure dark rate
pipeline.charge_generation.simple_dark_current.arguments.dark_rate = 0.01

# Disable CosmiX
pipeline.photon_generation.cosmix.enabled = False
 
# Run simulations and plot
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

### Science spectrum

In [ ]:
# Load a zeros array
pipeline.charge_generation.load_charge.arguments.filename = 'science_300s.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 300
exposure.readout.times = [300]

# Configure dark rate
pipeline.charge_generation.simple_dark_current.arguments.dark_rate = 0.01 

# Disable cosmix
pipeline.photon_generation.cosmix.enabled = False
 
# Run simulations and plot
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

### Cosmic rays

We use the inbuild CosmiX simulator to generate cosmic rays. This simulator uses a cosmic ray model with energy desposite and trails lengths provided from a dataset of protons penetrating a spacecraft shilded in a L2 orbit around solar maximum. Since the space environment is typically much more severe w.r.t. to the energy of cosmics, this model is a worst case example for our MARVEL simulations.

We use numpy's in-built Poisson random number generator to get the number of particles per second. The Poisson distribution is defined by 

$$ f(k;\lambda) = \frac{\lambda^k e^{-\lambda}}{k!} $$

For events with an expected separation $\lambda$ the Poisson distribution $ f(k;\lambda)$ describes the probability of $k$ events occurring within the observed interval $\lambda$.

The rate of cosmic rays reaching us falls off rapidly as the cosmic ray energy increases. If we only consider the more energetic cosmic rays (>1 GeV) the number of cosmic rays striking the Earth's atmosphere is a rate of 10,000 events/m$^2$/s. 

In [ ]:
# Configure CosmiX simulator to include cosmic rays (uses Gaia/PLATO L2 event model)
pipeline.photon_generation.cosmix.enabled = True
pipeline.photon_generation.cosmix.arguments.spectrum_file = '../../inputfiles/proton_L2_solarMax_11mm_Shielding.txt'

# Random number generator and seed
rng = np.random.default_rng()
pipeline.photon_generation.cosmix.arguments.seed = rng.integers(1e9, size=1)[0]

# Draw cosmic rays from Poisson distribution
ncosmics = rng.poisson(100)
rate     = ncosmics / 300    # Cosmics per 300 seconds
pipeline.photon_generation.cosmix.arguments.particles_per_second = rate

rate

In [ ]:
# Load a bias image
pipeline.charge_generation.load_charge.arguments.filename = 'bias_slow.fits'
pipeline.charge_generation.load_charge.arguments.time_scale = 900
exposure.readout.times = [900]

# Run simulations and plot (select Array = "image")
results = pyxel.exposure_mode(exposure, detector, pipeline)
pyxel.display_detector(detector)

___

## Configure YAML inputfile attributes

If you ever need to reconfigure or alter some attributes of Pyxel, the following configuration displays can be useful:

In [ ]:
pyxel.display_html(config)

In [ ]:
pyxel.display_html(detector)

In [ ]:
pyxel.display_html(pipeline)

In [ ]:
pyxel.display_html(pipeline.charge_transfer)

In [ ]:
exposure.readout